# Coding Attention Mechanisms

## Simple Version

In [1]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     x^1
    [0.55, 0.87, 0.66], # journey   x^2  
    [0.57, 0.86, 0.64], # starts    x^3
    [0.22, 0.58, 0.33], # with      x^4
    [0.77, 0.35, 0.10], # one       x^5
    [0.05, 0.80, 0.55]] # step      x^6
)

In [2]:
input_query = inputs[1]

In [3]:
input_1 = inputs[0]

In [4]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [5]:
i = 0

res = torch.dot(inputs[i], input_query)

print(res)

tensor(0.9544)


In [6]:
torch.empty(inputs.shape[0])

tensor([6.8610e+15, 1.2051e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [7]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4841, 0.8434, 0.7940, 1.0865])


In [8]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

In [9]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim = 0)

softmax_naive(attn_scores_2)

tensor([0.1369, 0.2351, 0.2326, 0.1225, 0.1166, 0.1563])

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)

In [11]:
query = inputs[1]

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4453, 0.6620, 0.5639])


## without trainable weights

In [12]:
attn_scores = torch.empty(6 ,6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9437, 0.4753, 0.4726, 0.6310],
        [0.9544, 1.4950, 1.4841, 0.8434, 0.7940, 1.0865],
        [0.9437, 1.4841, 1.4741, 0.8354, 0.8039, 1.0685],
        [0.4753, 0.8434, 0.8354, 0.4937, 0.4054, 0.6565],
        [0.4726, 0.7940, 0.8039, 0.4054, 0.7254, 0.3735],
        [0.6310, 1.0865, 1.0685, 0.6565, 0.3735, 0.9450]])


In [13]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9437, 0.4753, 0.4726, 0.6310],
        [0.9544, 1.4950, 1.4841, 0.8434, 0.7940, 1.0865],
        [0.9437, 1.4841, 1.4741, 0.8354, 0.8039, 1.0685],
        [0.4753, 0.8434, 0.8354, 0.4937, 0.4054, 0.6565],
        [0.4726, 0.7940, 0.8039, 0.4054, 0.7254, 0.3735],
        [0.6310, 1.0865, 1.0685, 0.6565, 0.3735, 0.9450]])

In [14]:
attn_weights = torch.softmax(attn_scores, dim = 1)
attn_weights

tensor([[0.2094, 0.2002, 0.1980, 0.1240, 0.1236, 0.1448],
        [0.1369, 0.2351, 0.2326, 0.1225, 0.1166, 0.1563],
        [0.1367, 0.2346, 0.2323, 0.1227, 0.1189, 0.1549],
        [0.1423, 0.2056, 0.2040, 0.1449, 0.1327, 0.1706],
        [0.1449, 0.1999, 0.2018, 0.1355, 0.1866, 0.1313],
        [0.1371, 0.2162, 0.2124, 0.1406, 0.1060, 0.1877]])

In [15]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4427, 0.6069, 0.5781],
        [0.4453, 0.6620, 0.5639],
        [0.4465, 0.6611, 0.5627],
        [0.4331, 0.6426, 0.5478],
        [0.4674, 0.6181, 0.5256],
        [0.4209, 0.6601, 0.5609]])

## Computing Attention Weights Step by Step with trainable weights

In [16]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [17]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))

In [18]:
query_2 = x_2 @ W_query

query_2

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)

In [19]:
keys = inputs @ W_key
value = inputs @ W_value

In [20]:
keys

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4379, 1.1229],
        [0.2408, 0.6706],
        [0.2011, 0.4019],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)

In [21]:
value

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3911, 0.9871],
        [0.2393, 0.5493],
        [0.1808, 0.3748],
        [0.3221, 0.7863]], grad_fn=<MmBackward0>)

In [22]:
key_2 = keys[1]
attn_scores_22 = torch.dot(query_2, key_2)

In [23]:
attn_scores_22

tensor(1.8524, grad_fn=<DotBackward0>)

In [24]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8225, 1.0795, 0.6714, 1.5440],
       grad_fn=<SqueezeBackward4>)

In [25]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2/ d_k ** 0.5, dim = -1)
attn_weights_2

tensor([0.1486, 0.2243, 0.2196, 0.1299, 0.0973, 0.1803],
       grad_fn=<SoftmaxBackward0>)

In [26]:
context_vec_2 = attn_weights_2 @ value

context_vec_2

tensor([0.3088, 0.8225], grad_fn=<SqueezeBackward4>)

## Implementing a Compact Self Attention Class

In [27]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = torch.nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = torch.nn.Parameter(torch.rand(d_in, d_out))


    def forward(self, x):
        queries = inputs @ self.W_query
        keys = inputs @ self.W_key
        value = inputs @ self.W_value

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/ d_k ** 0.5, dim = -1)
        context_vec = attn_weights @ value

        return context_vec
    
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
sa_v1.forward(inputs)

tensor([[0.3029, 0.8078],
        [0.3088, 0.8225],
        [0.3087, 0.8220],
        [0.2986, 0.7974],
        [0.2979, 0.7959],
        [0.3024, 0.8067]], grad_fn=<MmBackward0>)

In [28]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_query(inputs)
        keys = self.W_key(inputs)
        value = self.W_value(inputs)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/ d_k ** 0.5, dim = -1)
        context_vec = attn_weights @ value

        return context_vec

torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v2.forward(inputs)

tensor([[-0.5401, -0.1131],
        [-0.5386, -0.1160],
        [-0.5385, -0.1160],
        [-0.5360, -0.1156],
        [-0.5373, -0.1150],
        [-0.5362, -0.1161]], grad_fn=<MmBackward0>)

## Hiding Future Words with causal attention

In [29]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
value = sa_v2.W_value(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/ d_k ** 0.5, dim = -1)

In [30]:
attn_weights

tensor([[0.1715, 0.1760, 0.1760, 0.1554, 0.1634, 0.1577],
        [0.1633, 0.1746, 0.1745, 0.1609, 0.1619, 0.1649],
        [0.1633, 0.1746, 0.1744, 0.1609, 0.1619, 0.1649],
        [0.1635, 0.1702, 0.1701, 0.1650, 0.1640, 0.1673],
        [0.1656, 0.1720, 0.1719, 0.1622, 0.1638, 0.1646],
        [0.1622, 0.1707, 0.1705, 0.1652, 0.1634, 0.1680]],
       grad_fn=<SoftmaxBackward0>)

In [31]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [32]:
masked_simple = attn_weights * mask_simple
masked_simple

tensor([[0.1715, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1633, 0.1746, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1633, 0.1746, 0.1744, 0.0000, 0.0000, 0.0000],
        [0.1635, 0.1702, 0.1701, 0.1650, 0.0000, 0.0000],
        [0.1656, 0.1720, 0.1719, 0.1622, 0.1638, 0.0000],
        [0.1622, 0.1707, 0.1705, 0.1652, 0.1634, 0.1680]],
       grad_fn=<MulBackward0>)

In [33]:
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3188, 0.3408, 0.3405, 0.0000, 0.0000, 0.0000],
        [0.2444, 0.2545, 0.2543, 0.2467, 0.0000, 0.0000],
        [0.1982, 0.2059, 0.2058, 0.1941, 0.1960, 0.0000],
        [0.1622, 0.1707, 0.1705, 0.1652, 0.1634, 0.1680]],
       grad_fn=<DivBackward0>)


In [34]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.3111,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1655, 0.2602,   -inf,   -inf,   -inf,   -inf],
        [0.1653, 0.2595, 0.2584,   -inf,   -inf,   -inf],
        [0.0510, 0.1080, 0.1072, 0.0643,   -inf,   -inf],
        [0.1270, 0.1808, 0.1802, 0.0975, 0.1114,   -inf],
        [0.0476, 0.1192, 0.1182, 0.0731, 0.0580, 0.0966]],
       grad_fn=<MaskedFillBackward0>)


## Masking additional attention weights with dropout

In [35]:
torch.manual_seed(123)

layer = torch.nn.Dropout(0.5)

In [36]:
example = torch.ones(6, 6)
example

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

In [37]:
layer(attn_weights)

tensor([[0.3430, 0.3520, 0.0000, 0.3107, 0.3268, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3218, 0.0000, 0.3297],
        [0.3267, 0.3492, 0.3489, 0.3218, 0.0000, 0.3297],
        [0.0000, 0.3404, 0.3402, 0.0000, 0.0000, 0.3345],
        [0.0000, 0.3440, 0.0000, 0.3243, 0.0000, 0.3291],
        [0.0000, 0.3413, 0.3411, 0.3304, 0.3269, 0.0000]],
       grad_fn=<MulBackward0>)

## Compact Causal Self-attention mask

In [38]:
batch = torch.stack((inputs, inputs), dim = 0)
batch.shape

torch.Size([2, 6, 3])

In [39]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias = False):
        super().__init__()
        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        value = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores/ keys.shape[-1] ** 0.5, dim = -1)
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ value

        return context_vec

torch.manual_seed(789)
context_length = batch.shape[1]
dropout = 0.0
ca = CausalAttention(d_in, d_out, context_length, dropout)
ca.forward(batch)

tensor([[[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.1004,  0.0631],
         [-0.0986,  0.0487],
         [-0.0550,  0.1073],
         [-0.0786,  0.0670]],

        [[-0.0872,  0.0286],
         [-0.0991,  0.0501],
         [-0.1004,  0.0631],
         [-0.0986,  0.0487],
         [-0.0550,  0.1073],
         [-0.0786,  0.0670]]], grad_fn=<UnsafeViewBackward0>)

## Multi Head Attention

In [40]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads = 2):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out, context_length, dropout) for _ in range(num_heads)
            ])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim = -1)

torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, dropout=0.0, num_heads = 2)
mha.forward(batch)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6313, -0.0648,  0.6213,  0.3878],
         [-0.5684, -0.0854,  0.5485,  0.3602],
         [-0.5601, -0.1081,  0.5383,  0.3540],
         [-0.5362, -0.1161,  0.5130,  0.3586]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6313, -0.0648,  0.6213,  0.3878],
         [-0.5684, -0.0854,  0.5485,  0.3602],
         [-0.5601, -0.1081,  0.5383,  0.3540],
         [-0.5362, -0.1161,  0.5130,  0.3586]]], grad_fn=<CatBackward0>)

## Multi Head Attention with Split Weights

In [42]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forwar

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 4
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0155,  0.3260, -0.0734, -0.3709],
         [-0.0122,  0.3139, -0.0708, -0.3616],
         [-0.0169,  0.2987, -0.0694, -0.3466],
         [-0.0175,  0.3008, -0.0686, -0.3428]],

        [[ 0.1184,  0.3120, -0.0847, -0.5774],
         [ 0.0178,  0.3221, -0.0763, -0.4225],
         [-0.0155,  0.3260, -0.0734, -0.3709],
         [-0.0122,  0.3139, -0.0708, -0.3616],
         [-0.0169,  0.2987, -0.0694, -0.3466],
         [-0.0175,  0.3008, -0.0686, -0.3428]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])
